In [51]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import transforms
import pandas as pd
from PIL import Image
import math
import pickle

In [52]:
LR = 0.05
BATCH_SIZE = 64
EPOCH = 100
DATA_FOLDER = 'set03'

In [53]:
Train_Loss_d = []
Train_Loss_phi = []
Train_Loss = []
Test_Loss_d = []
Test_Loss_phi = []
Test_Loss = []

In [54]:
def getData(mode):
    if mode == 'train':
        
        img = pd.read_csv(DATA_FOLDER + '_train_img.csv')
        label = pd.read_csv(DATA_FOLDER + '_train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv(DATA_FOLDER + '_test_img.csv')
        label = pd.read_csv(DATA_FOLDER + '_test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)


class Loader(Data.Dataset):
    def __init__(self, root, mode):
        """
        Args:
            root (string): Root path of the dataset.
            mode : Indicate procedure status(training or testing)

            self.img_name (string list): String list that store all image names.
            self.label (int or float list): Numerical list that store all ground truth label values.
        """
        self.root = root
        self.img_name, self.label = getData(mode)
        self.mode = mode
        print("> Found %d images..." % (len(self.img_name)))
        

    def __len__(self):
        #------------return the size of dataset
        return len(self.img_name)

    def __getitem__(self, index):
        #-------------Get the image path from 'self.img_name' and load it.
                  
        path = self.root + str(self.img_name[index]) + '.png'
        img_as_img = Image.open(path)
        #img = img.resize(,Image.ANTIALIAS)
        #box=(0,240,640,480)
        #img=img.crop(box)
        
        #-------------Get the ground truth label from self.label"""
        label = torch.from_numpy(self.label)[index]
        
        #-------------Transform the .jpeg rgb images
        if self.mode == 'train':
            transform1 = transforms.Compose([
#                 transforms.RandomHorizontalFlip(p=0.5),
#                 transforms.RandomVerticalFlip(p=0.5),
#                 transforms.RandomRotation(degrees=(-45,45), resample=False, expand=False, center=None),
#                 transforms.ColorJitter(contrast=(0,1)),
                transforms.Resize(( 280, 210)),
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                #transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)),#range [0, 255] ->[-1.0,1.0]
                
                ]
            )
        else:
            transform1 = transforms.Compose([
                transforms.Resize(( 280, 210)),
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                #transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)),
             
                ]
            )
        img_tran = transform1(img_as_img)
                
        #-------------Return processed image and label
        return img_tran, label

In [55]:
# Dataloader

train_data=Loader('./'+ DATA_FOLDER +'/','train')
test_data=Loader('./'+ DATA_FOLDER +'/','test')
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE)
test_loader = Data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
print (train_loader)
print (test_loader)

> Found 2652 images...
> Found 294 images...


In [56]:

class DeepConvNet(nn.Module):
    def __init__(self):
        super(DeepConvNet, self).__init__()
    
        #####---1
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0, bias=False)
        self.batchnorm1 = nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.norm1 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        #####
        
        #####---2
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=(2, 2) ,bias=True, groups=2)
        self.batchnorm2 = nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.norm2 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        #####
        
        #####---3
        self.conv3 = nn.Conv2d(256, 384, kernel_size=(3, 3), padding=(1, 1), bias=False)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=(3, 3), padding=(1, 1), bias=True, groups=2)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=(3, 3), padding=(1, 1), bias=True, groups=2)
        self.pool5 = nn.MaxPool2d(kernel_size=3 , stride=2, padding=0)
        #####
        
        self.fc6 = nn.Linear(in_features=8960, out_features=4096, bias=True)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.fc8 = nn.Linear(in_features=4096, out_features=256, bias=False)
        self.fc9 = nn.Linear(in_features=256, out_features=2, bias=True)
        
    def forward(self, x):
        
        ###-----1------
        out = self.conv1(x)
        #out = self.batchnorm1(out)
        out = F.relu(out)
        out = self.pool1(out)        
        out = self.norm1(out)
        #out = F.dropout(out, p=0.5)
        
        ###-----2------
        out = self.conv2(out)
        #out = self.batchnorm2(out)
        out = F.relu(out)
        out = self.pool2(out)
        out = self.norm2(out)
        #out = F.dropout(out, p=0.5)
        
        ###-----3------
        out = self.conv3(out)
        out = F.relu(out)
        ###-----4------
        out = self.conv4(out)
        out = F.relu(out)
        
        out = self.conv5(out)
        out = F.relu(out)
        out = self.pool5(out)
        
        out = out.view(out.size(0), -1) #flatten
        out = self.fc6(out)
        out = F.leaky_relu(out,0.05)
        out = F.dropout(out, p=0.5)
        
        out = self.fc7(out)
        out = F.leaky_relu(out,0.05)
        out = F.dropout(out, p=0.5)
      
        out = self.fc8(out)
        out = F.leaky_relu(out,0.05)
        out = F.dropout(out, p=0.5)
        
        out = self.fc9(out)
        #out = torch.sigmoid(out)
        #out = torch.tanh(out)
        #out = F.softsign(out)
        
        
        return out

In [57]:
def train_DeepConv(epoch):
    model_DeepConvNet.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)

        optimizer.zero_grad()
        output = model_DeepConvNet(data)
        target = target.float()
        loss_d = Loss(output[0], target[0])
        loss_phi = Loss(output[1], target[1])
        loss = loss_d + loss_phi
        loss.backward()
        optimizer.step()
#         print("output" ,output.cpu().detach().numpy().squeeze()[0:3])
#         print("ground truth" ,target.cpu().detach().numpy().squeeze()[0:3])
    Train_Loss.append(round((loss.data).cpu().numpy().tolist(),6))
    Train_Loss_d.append(round((loss_d.data).cpu().numpy().tolist(),6))
    Train_Loss_phi.append(round((loss_phi.data).cpu().numpy().tolist(),6))
   
    print('Train Epoch: {} \t Loss_d: {} Loss_phi:{}'.format(
            epoch, loss_d.data , loss_phi.data))

In [58]:
def test_DeepConv(epoch):
    model_DeepConvNet.eval()
    test_loss_d = 0.0
    test_loss_phi = 0.0
    
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)
        with torch.no_grad():
            output = model_DeepConvNet(data)
        target = target.float()
        test_loss_d = Loss(output[0], target[0])
        test_loss_phi = Loss(output[1], target[1])
    
    if(epoch==1):
        print(" ground truth\n" ,target.cpu().detach().numpy().squeeze()[0:8])
    print(output.cpu().detach().numpy().squeeze()[0:8])
    
    Test_Loss_d.append(round((test_loss_d.data).cpu().numpy().tolist(),6))
    Test_Loss_phi.append(round((test_loss_phi.data).cpu().numpy().tolist(),6))
        
    print('========Test set: Average loss d:{}\t loss phi:{}========\n'
          .format(test_loss_d,test_loss_phi))

In [59]:
model_DeepConvNet = DeepConvNet().cuda(0)

if torch.cuda.is_available():
    device = torch.device('cuda')
    model_DeepConvNet.to(device) 
#------------define optimizer/loss function
Loss = nn.MSELoss(reduction='mean')    
optimizer = torch.optim.SGD(model_DeepConvNet.parameters(), lr=LR, momentum=0.9, 
                            dampening=0, weight_decay=0.0005, nesterov=False)
print(model_DeepConvNet)

DeepConvNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), bias=False)
  (batchnorm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
  (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (pool5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilati

In [60]:
#---------------Train-----------------
for epoch in range(1,EPOCH+1):
    train_DeepConv(epoch)
    test_DeepConv(epoch)
    if (epoch%10==0):
        PATH ='./model/'+str(epoch)+'-linear.pkl'
        torch.save(model_DeepConvNet.state_dict(), PATH)
    


Train Epoch: 1 	 Loss_d: 0.025751203298568726 Loss_phi:0.024926993995904922
 ground truth
 [[-0.08273652 -0.94172   ]
 [-0.11888251  0.37627342]
 [ 0.05218287  0.7489732 ]
 [-0.09701227  0.3416342 ]
 [-0.02468774  0.07899258]
 [ 0.00708377 -0.32711706]
 [-0.02614982  0.09217016]
 [-0.03992722  0.04558269]]
[[ 0.08923959 -0.1495622 ]
 [ 0.0868092  -0.13917881]
 [ 0.08068758 -0.14746723]
 [ 0.08472417 -0.14108004]
 [ 0.08816658 -0.14835301]
 [ 0.09156924 -0.14135914]
 [ 0.09023021 -0.13693337]
 [ 0.0897599  -0.15014556]]
========Test set: Average loss d:0.32854488492012024	 loss phi:0.15400005877017975========

Train Epoch: 2 	 Loss_d: 0.024938959628343582 Loss_phi:0.022831931710243225
[[ 0.08579266 -0.14122088]
 [ 0.0909816  -0.13661613]
 [ 0.08419894 -0.15037055]
 [ 0.07615604 -0.13969015]
 [ 0.09048502 -0.14063579]
 [ 0.09220293 -0.13511306]
 [ 0.07946342 -0.13450846]
 [ 0.08544393 -0.1425368 ]]
========Test set: Average loss d:0.3346004784107208	 loss phi:0.15354932844638824========


Train Epoch: 22 	 Loss_d: 0.024899644777178764 Loss_phi:0.02468346245586872
[[ 0.0786642  -0.15033308]
 [ 0.07831585 -0.15085922]
 [ 0.07884221 -0.14984497]
 [ 0.07781729 -0.15178797]
 [ 0.07866767 -0.15016241]
 [ 0.07933712 -0.1501611 ]
 [ 0.07876178 -0.15021186]
 [ 0.07777597 -0.15199858]]
========Test set: Average loss d:0.32617175579071045	 loss phi:0.158378005027771========

Train Epoch: 23 	 Loss_d: 0.025266526266932487 Loss_phi:0.02500849775969982
[[ 0.07926783 -0.15020552]
 [ 0.07839257 -0.15261303]
 [ 0.07896414 -0.15004398]
 [ 0.07807897 -0.15251403]
 [ 0.079059   -0.1504118 ]
 [ 0.07920188 -0.15036514]
 [ 0.07888713 -0.14983742]
 [ 0.07944339 -0.15076743]]
========Test set: Average loss d:0.32637032866477966	 loss phi:0.15931916236877441========

Train Epoch: 24 	 Loss_d: 0.02489009127020836 Loss_phi:0.02474994957447052
[[ 0.07873761 -0.150365  ]
 [ 0.07844493 -0.15025331]
 [ 0.0780946  -0.15033473]
 [ 0.07828692 -0.15017764]
 [ 0.07861119 -0.14975427]
 [ 0.07801805 -0.15025

KeyboardInterrupt: 

In [61]:
# pickle a variable to a file
file = open('Train_loss_d.pickle', 'wb')
pickle.dump(Train_Loss_d, file)
file.close()
# pickle a variable to a file
file = open('Train_loss_phi.pickle', 'wb')
pickle.dump(Train_Loss_phi, file)
file.close()
# pickle a variable to a file
file = open('Train_loss.pickle', 'wb')
pickle.dump(Train_Loss, file)
file.close()
# pickle a variable to a file
file = open('Test_loss_d.pickle', 'wb')
pickle.dump(Test_Loss_d, file)
file.close()
# pickle a variable to a file
file = open('Test_loss_phi.pickle', 'wb')
pickle.dump(Test_Loss_phi, file)
file.close()